In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from matplotlib.animation import FuncAnimation, HTMLWriter
from IPython.core.display import HTML

In [2]:
def gauss(x, a):
    return (2 * a / np.pi)**(1/4) * np.exp(-a * x**2)

# Model elektronu w sieci krystalicznej

In [7]:
dt = 0.005 * 10**(-15)
dx = 0.1 * 10**(-9)
hbar = 1.054 * 10**(-34)
m = 0.91 * 10**(-30)

class Psi:
    ra = (hbar/(2*m)) * (dt/(dx**2))
    a = 2.0
    Re = []
    Im = []
    Im_r = []
    Im_f = []
    P = []
    X = []
    V = []
    def initialise(safe):
        x = -15
        while(x <= 15):
            Psi.Re.append(gauss(x, Psi.a))
            Psi.Im.append(0.0)
            Psi.Im_r.append(0.0)
            Psi.Im_f.append(0.0)
            Psi.V.append(0.0) 
            Psi.P.append(gauss(x, Psi.a)**2)
            Psi.X.append(x)
            x += 0.025 
    def update(safe, n):
        Re = Psi.Re
        Im = Psi.Im
        for k in range(0, n):
            for i in range(0, len(Psi.Re) - 1):
                Psi.Re[i] = Re[i] - Psi.ra * (Im[i + 1] - 2 * Im[i] + Im[i - 1]) + dt/hbar * Psi.V[i] * Im[i]
                Psi.Im[i] = Im[i] + Psi.ra * (Re[i + 1] - 2 * Re[i] + Re[i - 1]) - dt/hbar * Psi.V[i] * Re[i]               
                Psi.Im_r[i] = Im[i]
                Psi.Im_f[i] = Psi.Im[i] + Psi.ra * (Psi.Re[i + 1] - 2 * Psi.Re[i] + Psi.Re[i - 1]) - dt/hbar * Psi.V[i] * Re[i] 
                Psi.P[i] = Psi.Re[i]**2 + Psi.Im_r[i] * Psi.Im_f[i]
    def set_potential(safe, V):
        for i in range(0, len(Psi.V)):
            Psi.V[i] = V[i] 
    def set_potential(safe, V):
        for i in range(0, len(Psi.V)):
            Psi.V[i] = V[i]
    def compute_energy(safe):
        A = hbar**2 / (2 * m * dx**2)
        ke = 0 + 0.j
        pe = 0 + 0.j
        psi = []
        for i in range(0, len(Psi.Re)):
            psi.append(Psi.Re[i] + Psi.Im[i] * 1.j)
            pe += psi[i] * psi[i].conjugate() * Psi.V[i]
        for i in range(1, len(Psi.Re) - 1):
            Lap = psi[i + 1] - 2 * psi[i] + psi[i - 1]
            ke += Lap * psi[i].conjugate() * (Psi.X[i] - Psi.X[i - 1])
        KE = -A * np.real(ke)/(1.6 * 10**(-19))
        PE = np.real(pe)
        return (PE + KE)       
       
    

In [8]:
  # Definicja potencjału
V = []
v = []
x = -15
while(x <= 15):
    if((x >= -10.04 and x <= -9.96) or (x >= -6.04 and x <= -5.96) or (x >= -2.04 and x <= -1.96) or (x >= 1.96 and x <= 2.04) or (x >= 5.96 and x <= 6.04) or (x >= 9.96 and x <= 10.04)):
        V.append(0.5 * 1.6 * 10**(-19))
        v.append(1.5)
    else:
        V.append(0.0)
        v.append(0)
    x += 0.025  

In [9]:
psi = Psi()
psi.initialise()
psi.set_potential(V)
psi.compute_energy()

0.004767208581146815

In [11]:
fig = plt.figure(figsize = (18,8))
line4, = plt.plot([], [],'C4')
line1, = plt.plot([], [],'C0')
line2, = plt.plot([], [],'C1')
line3, = plt.plot([], [],'C2')

plt.grid(linestyle = '--',linewidth = 1)
plt.ylim(-0.75,1.5)
plt.xlim(-12, 12)
plt.xlabel('$x$')
plt.ylabel('$\Psi(x,t)$')

plt.close()

def frame(i):
    line4.set_data(psi.X, v)
    line1.set_data(psi.X, psi.Re)
    line2.set_data(psi.X, psi.Im)
    line3.set_data(psi.X, psi.P)    
    if(i > 0):
        psi.update(100)
    
    return line1, line2, line3



animation = FuncAnimation(fig, frame, frames = 1, interval = 20)

#file = r"grzebyk.html"
#writervideo = anim.HTMLWriter(fps=100) 
#animation.save(file, writer=writervideo)

HTML(animation.to_jshtml())